# EDA Part 2

In [1]:
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import re
import datetime
import altair as alt

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

In [2]:
# Check database
with sqlite3.connect('./pre_processed_data/lab2.db') as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    
print(cursor.fetchall())
conn.close()

[('artist',), ('billboard_old',), ('spotify_orig',), ('spotify_orig_v2',), ('billboard_back2',), ('wiki_artists',), ('spotify',), ('spotify2',), ('billboard_backup',), ('billboard_artists',), ('billboard',), ('billboard_back3',)]


## 1. Query Wikipedia

In [3]:
# Get Wikipedia artists from database
with sqlite3.connect('./pre_processed_data/lab2.db') as conn:
        q = '''SELECT * FROM wiki_artists
            ''' 
df_wiki = pd.read_sql(q, conn)
conn.close()

In [4]:
# Check artist names
df_wiki['name'].unique()[10000:11000]

array(['hoàng quyên', 'lynda trang Đài', 'phạm lương thanh trúc',
       'hoàng hải', 'trần văn khê', 'ha okio', 'big baby gandhi',
       'monica yunus', 'bua xou mua', 'kesang marstrand', 'angel tang',
       'alex wong', 'agnes lum', 't.v. carpio', 'larissa lam',
       'peter moon', 'lee pui ming', 'magdalen hsu-li', 'david oei',
       'sam tsui', 'rita porfiris', 'masaki liu', 'jason kao hwang',
       'auliʻi cravalho', 'charles mingus', 'malay', 'huang ruo',
       'pete miser', 'chen yihan', 'kaiser kuo', 'freddie wong',
       'catalina yue', 'jadin wong', 'tia carrere', 'carrie ann inaba',
       'gabriela lena frank', 'ge wang', 'greta kline', 'joe wong', 'zhu',
       'chloe bennet', 'hao huang', 'yuhua hamasaki', 'cory wong',
       'alfred apaka', 'john bruce yeh', 'carmit bachar', 'chi cheng',
       'evelina chao', 'kelis', 'dai-keong lee', 'alan chang',
       'adam fong', 'dorothy chang', 'naomi yang', 'nicholas phan',
       'peter mui', 'ben fong-torres', 'leah diz

In [5]:
# Check artist country
df_wiki['country'].unique()

array(['South Korean', 'Bruneian', 'Burmese', 'Cambodian', 'Chinese',
       'East Timorese', 'Filipino', 'Indonesian', 'Japanese', 'Laotian',
       'Malaysian', 'Mongolian', 'Singaporean', 'Thai', 'Vietnamese',
       'Bangladeshi American', 'Laotian American', 'Tibetan American',
       'Chinese American', 'Filipino American', 'Indian American',
       'Indonesian American', 'Japanese American', 'Korean American',
       'Pakistani American', 'Taiwanese American'], dtype=object)

## 2. Query Billboard

In [6]:
# Get Billboard charts results
with sqlite3.connect('./pre_processed_data/lab2.db') as conn:
        q = '''SELECT * FROM billboard
            ''' 
df_billboard = pd.read_sql(q, conn)
conn.close()

In [7]:
# Check the query
df_billboard.head()

,chartname,date,song,artist,rank
0,weekly_hot_100,2011-01-03,Grenade,bruno mars,1
1,weekly_hot_100,2011-01-03,Firework,katy perry,2
2,weekly_hot_100,2011-01-03,We R Who We R,ke$ha,3
3,weekly_hot_100,2011-01-03,What's My Name?,rihanna,4
4,weekly_hot_100,2011-01-03,Raise Your Glass,p!nk,5


In [8]:
# Check the charts that were scraped
df_billboard['chartname'].unique()

array(['weekly_hot_100', 'weekly_billboard_200',
       'weekly_billboard_global_200',
       'weekly_billboard_global_exclUS_200'], dtype=object)

In [9]:
# Create the month_year and Asian column
df_billboard['month_yr'] = pd.to_datetime(
    df_billboard['date']).dt.to_period('M')
df_billboard["asian"] = df_billboard["artist"].isin(df_wiki['name'])

df_billboard.head()

,chartname,date,song,artist,rank,month_yr,asian
0,weekly_hot_100,2011-01-03,Grenade,bruno mars,1,2011-01,True
1,weekly_hot_100,2011-01-03,Firework,katy perry,2,2011-01,False
2,weekly_hot_100,2011-01-03,We R Who We R,ke$ha,3,2011-01,False
3,weekly_hot_100,2011-01-03,What's My Name?,rihanna,4,2011-01,False
4,weekly_hot_100,2011-01-03,Raise Your Glass,p!nk,5,2011-01,False


In [10]:
# Combine billboard table and Wiki table
df_billboard = pd.merge(df_billboard, df_wiki, 
                        left_on='artist', right_on='name', how="left")

In [11]:
# Create a subset of the Weekly Hot 100 chart
df_hot_100 = df_billboard[df_billboard['chartname'] == 'weekly_hot_100']

In [12]:
# Create a subset of the Billboard Global 200
df_global_bb = (df_billboard[df_billboard['chartname'] == 
                             'weekly_billboard_global_200'])

df_global_bb = df_global_bb[df_global_bb['rank'] <= 100]

## 3. Query Spotify 

In [13]:
# Get Spotify charts results
with sqlite3.connect('./pre_processed_data/lab2.db') as conn:
        q = '''SELECT * FROM spotify
            ''' 
df_spotify = pd.read_sql(q, conn)
conn.close()

In [14]:
# Check the query obtained
df_spotify.head()

,chartname,chartfreq,date,region,rank,song,artist,artist2,stream_count
0,viral50,daily,2017-01-05,global,1,Sigurado,by Up Dharma Down,up dharma down,0
1,viral50,daily,2017-01-05,global,2,Fill Me In - AFTR:HRS Mixcut,"by Loud Luxury, Ryan Shepherd",loud luxury,0
2,viral50,daily,2017-01-05,global,2,Fill Me In - AFTR:HRS Mixcut,"by Loud Luxury, Ryan Shepherd",ryan shepherd,0
3,viral50,daily,2017-01-05,global,3,Losing Sleep,by Vorsa,vorsa,0
4,viral50,daily,2017-01-05,global,4,Deu Onda,by MC G15,mc g15,0


In [15]:
# Create new column for month_yr and Asians
df_spotify['month_yr'] = pd.to_datetime(df_spotify['date']).dt.to_period('M')
df_spotify["asian"] = df_spotify["artist2"].isin(df_wiki['name'])

In [16]:
# Combine the dataframe with the Wikipedia data
df_spotify2 = pd.merge(df_spotify, df_wiki, 
                       left_on='artist2', right_on='name', how='left')
df_spotify2

,chartname,chartfreq,date,region,rank,song,artist,artist2,stream_count,month_yr,asian,name,country,lineage
0,viral50,daily,2017-01-05,global,1,Sigurado,by Up Dharma Down,up dharma down,0,2017-01,False,NaN,NaN,NaN
1,viral50,daily,2017-01-05,global,2,Fill Me In - AFTR:HRS Mixcut,"by Loud Luxury, Ryan Shepherd",loud luxury,0,2017-01,False,NaN,NaN,NaN
2,viral50,daily,2017-01-05,global,2,Fill Me In - AFTR:HRS Mixcut,"by Loud Luxury, Ryan Shepherd",ryan shepherd,0,2017-01,False,NaN,NaN,NaN
3,viral50,daily,2017-01-05,global,3,Losing Sleep,by Vorsa,vorsa,0,2017-01,False,NaN,NaN,NaN
4,viral50,daily,2017-01-05,global,4,Deu Onda,by MC G15,mc g15,0,2017-01,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241745,top200,weekly,2021-07-23,us,196,Hold On,by Justin Bieber,justin bieber,1419124,2021-07,False,NaN,NaN,NaN
1241746,top200,weekly,2021-07-23,us,197,I Like That,by Bazzi,bazzi,1418767,2021-07,False,NaN,NaN,NaN
1241747,top200,weekly,2021-07-23,us,198,21,by Polo G,polo g,1410427,2021-07,False,NaN,NaN,NaN
1241748,top200,weekly,2021-07-23,us,199,Heartless,by Kanye West,kanye west,1409485,2021-07,False,NaN,NaN,NaN


In [17]:
# Obtain a subset of the Spotify Top 200 Global charts
df_spotify_top200_global = df_spotify2[df_spotify2['chartname'] == 'top200']
df_spotify_top200_global = df_spotify_top200_global[
    df_spotify_top200_global['chartfreq'] == 'weekly']
df_spotify_top200_global = df_spotify_top200_global[
    df_spotify_top200_global['region'] == 'global']
df_spotify_top200_global = df_spotify_top200_global[
    df_spotify_top200_global['rank'] <= 100]
df_spotify_top200_global.head()

,chartname,chartfreq,date,region,rank,song,artist,artist2,stream_count,month_yr,asian,name,country,lineage
1118434,top200,weekly,2016-12-23,global,1,Starboy,"by The Weeknd, Daft Punk",the weeknd,25286465,2016-12,False,NaN,NaN,NaN
1118435,top200,weekly,2016-12-23,global,1,Starboy,"by The Weeknd, Daft Punk",daft punk,25286465,2016-12,False,NaN,NaN,NaN
1118436,top200,weekly,2016-12-23,global,2,Closer,"by The Chainsmokers, Halsey",the chainsmokers,22047697,2016-12,False,NaN,NaN,NaN
1118437,top200,weekly,2016-12-23,global,2,Closer,"by The Chainsmokers, Halsey",halsey,22047697,2016-12,False,NaN,NaN,NaN
1118438,top200,weekly,2016-12-23,global,3,Rockabye (feat. Sean Paul & Anne-Marie),by Clean Bandit,clean bandit,19794482,2016-12,False,NaN,NaN,NaN


In [18]:
# Obtain a subset of the Spotify Top 200 US charts
df_spotify_top200_us = df_spotify2[df_spotify2['chartname'] == 'top200']
df_spotify_top200_us = df_spotify_top200_us[
    df_spotify_top200_us['chartfreq'] == 'weekly']
df_spotify_top200_us = df_spotify_top200_us[
    df_spotify_top200_us['region'] == 'us']
df_spotify_top200_us = df_spotify_top200_us[
    df_spotify_top200_us['rank'] <= 100]
df_spotify_top200_us

,chartname,chartfreq,date,region,rank,song,artist,artist2,stream_count,month_yr,asian,name,country,lineage
1183967,top200,weekly,2016-12-23,us,1,Fake Love,by Drake,drake,8618267,2016-12,False,NaN,NaN,NaN
1183968,top200,weekly,2016-12-23,us,2,Bad and Boujee (feat. Lil Uzi Vert),by Migos,migos,7946032,2016-12,False,NaN,NaN,NaN
1183969,top200,weekly,2016-12-23,us,3,Starboy,"by The Weeknd, Daft Punk",the weeknd,7738517,2016-12,False,NaN,NaN,NaN
1183970,top200,weekly,2016-12-23,us,3,Starboy,"by The Weeknd, Daft Punk",daft punk,7738517,2016-12,False,NaN,NaN,NaN
1183971,top200,weekly,2016-12-23,us,4,Black Beatles,"by Rae Sremmurd, Gucci Mane",rae sremmurd,6209336,2016-12,False,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1241634,top200,weekly,2021-07-23,us,96,"Whats Poppin (feat. DaBaby, Tory Lanez & Lil W...",by Jack Harlow,jack harlow,1945998,2021-07,False,NaN,NaN,NaN
1241635,top200,weekly,2021-07-23,us,97,ROCKSTAR (feat. Roddy Ricch),by DaBaby,dababy,1945182,2021-07,False,NaN,NaN,NaN
1241636,top200,weekly,2021-07-23,us,98,Stoned at the Nail Salon,by Lorde,lorde,1942072,2021-07,False,NaN,NaN,NaN
1241637,top200,weekly,2021-07-23,us,99,"1 step forward, 3 steps back",by Olivia Rodrigo,olivia rodrigo,1936744,2021-07,True,olivia rodrigo,Filipino American,Mixed Asian


In [19]:
# Create a new column to get unique chart name
df_spotify_top200_us['chart_name'] = (
    df_spotify_top200_us['chartname'] +
    '_' +
    df_spotify_top200_us['region'])
    
df_spotify_top200_global['chart_name'] = (
    df_spotify_top200_global['chartname'] +
    '_' +
    df_spotify_top200_global['region'])
    
df_hot_100['chart_name'] = df_hot_100['chartname']
df_global_bb['chart_name'] = df_global_bb['chartname']

<ipython-input-19-82659c5da919>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_hot_100['chart_name'] = df_hot_100['chartname']


## Visualization Preparation

In [20]:
# Combine all charts into one DataFrame
charts = [df_spotify_top200_us, df_spotify_top200_global,
          df_hot_100, df_global_bb]
chart_names = ['Spotify Top 200 US', 'Spotify Top 200 Global', 
                     'Billboard Hot 100', 'Billboard Global 200']
df_charts = pd.DataFrame()

for i in charts:
    new_df = (i.groupby(['month_yr', 'chart_name'])['asian'].sum()
              .to_frame())
    df_charts = pd.concat([df_charts, new_df], axis=1)
    
df_charts.columns = chart_names
df_charts.reset_index(inplace=True)
df_charts.drop(columns=['chart_name'], inplace=True)
df_charts.set_index('month_yr')
df_charts['month_yr'] = df_charts['month_yr'].astype(str)

In [21]:
# Prepare DataFrame for plotting
df_charts_melt = df_charts[df_charts['month_yr'] >= '2016-12']
df_charts_melt = df_charts_melt.melt(
    id_vars='month_yr', value_vars=chart_names).dropna()
df_charts_melt.head()

,month_yr,variable,value
1,2016-12,Spotify Top 200 US,10.0
4,2017-01,Spotify Top 200 US,16.0
7,2017-02,Spotify Top 200 US,20.0
10,2017-03,Spotify Top 200 US,19.0
13,2017-04,Spotify Top 200 US,13.0


In [22]:
# Save DataFrame to csv
df_charts_melt.to_csv('./post_processed_data/counts_over_time.csv')

In [23]:
# Creates a plot of Asian artists count in the charts over time
df_charts_melt = pd.read_csv('./post_processed_data/counts_over_time.csv')

chart_line = alt.Chart(df_charts_melt, 
                       title="Asians in Different Music Charts through Time"
).mark_line().encode(
    x=alt.X('month_yr', axis=alt.Axis(title='Date')),
    y=alt.Y('value', scale=alt.Scale(domain=[0, 70]),
            axis=alt.Axis(title='Number of Time in Chart')),
    color=alt.Color('variable', legend=alt.Legend(title="Music Chart")),
    strokeDash='variable',
).properties(
    width=800,
    height=300
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=18,
    offset=5,                          
    orient='top',
    anchor='middle')

chart_line

# https://altair-viz.github.io/user_guide/customization.html

alt.Chart(...)

In [24]:
# Combine the DataFrame in terms of lineage
df_charts_orig = pd.DataFrame()

for i in charts:
    lineage_count = i.groupby(['month_yr', 'lineage']).size().to_frame()
    df_charts_orig = pd.concat([df_charts_orig, lineage_count], axis=1)
    
df_charts_orig.columns = chart_names
df_charts_orig = (df_charts_orig.reset_index()
                  .set_index('month_yr').loc['2020-09':])
df_charts_orig.head()

,lineage,Spotify Top 200 US,Spotify Top 200 Global,Billboard Hot 100,Billboard Global 200
month_yr,,,,,
2020-09,Asian,26.0,20.0,4.0,7.0
2020-09,Mixed Asian,4.0,4.0,4.0,2.0
2020-10,Asian,24.0,38.0,5.0,32.0
2020-10,Mixed Asian,6.0,5.0,4.0,4.0
2020-11,Asian,17.0,29.0,11.0,53.0


In [25]:
# Prepare the DataFrame for plotting
df_orig_melt = df_charts_orig.reset_index().melt(id_vars=['month_yr', 
                                                          'lineage'])
df_orig_melt['month_yr'] = df_orig_melt['month_yr'].astype(str)
df_orig_melt

,month_yr,lineage,variable,value
0,2020-09,Asian,Spotify Top 200 US,26.0
1,2020-09,Mixed Asian,Spotify Top 200 US,4.0
2,2020-10,Asian,Spotify Top 200 US,24.0
3,2020-10,Mixed Asian,Spotify Top 200 US,6.0
4,2020-11,Asian,Spotify Top 200 US,17.0
5,2020-11,Mixed Asian,Spotify Top 200 US,4.0
6,2020-12,Asian,Spotify Top 200 US,6.0
7,2020-12,Mixed Asian,Spotify Top 200 US,4.0
8,2021-01,Asian,Spotify Top 200 US,12.0
9,2021-01,Mixed Asian,Spotify Top 200 US,11.0


In [26]:
# Save DataFrame to csv
df_orig_melt.to_csv('./post_processed_data/lineage_over_time.csv')

In [27]:
# Plot of Asians vs Mixed-Asians through Time
stacked_bar = alt.Chart(df_orig_melt,
                        title="Asians and Mixed-Asians through time"
).mark_bar().encode(
    x=alt.X('month_yr:N', title=None),
    y=alt.Y('sum(value):Q', axis=alt.Axis(grid=False,
                                          title='Number of Time in Chart')),
    column=alt.Column('variable:N', title=None),
    color=alt.Color('lineage:N', 
                    scale=alt.Scale(range=['#D9734E', '#F2B05E']))
).configure_view(
    strokeOpacity=0    
).properties(
    width=180,
    height=300
).configure_axis(
    labelFontSize=12,
    titleFontSize=14
).configure_title(
    fontSize=18, 
    offset=5,                                  
    orient='top', 
    anchor='middle')

stacked_bar

# https://github.com/altair-viz/altair/issues/1679

alt.Chart(...)

## 5. Insights Exploration

In [28]:
# Check spotify top 200 global
sp_g = df_spotify_top200_global.groupby(
    ['month_yr', 'artist2'])['asian'].sum().to_frame()
sp_g = sp_g[sp_g['asian'] != 0].reset_index().set_index('month_yr') 

In [29]:
sp_g.loc['2019-04']

,artist2,asian
month_yr,,
2019-04,blackpink,5
2019-04,bruno mars,3
2019-04,bts,9
2019-04,dominic fike,1


In [30]:
# Check spotify top 200 us
sp_us = df_spotify_top200_us.groupby(
    ['month_yr', 'artist2'])['asian'].sum().to_frame()
sp_us = sp_us[sp_us['asian'] != 0].reset_index().set_index('month_yr') 

In [31]:
sp_us.loc['2021-04']

,artist2,asian
month_yr,,
2021-04,anderson .paak,5
2021-04,bruno mars,9
2021-04,conan gray,3
2021-04,h.e.r.,1
2021-04,olivia rodrigo,10


In [32]:
df_hot_100[df_hot_100['artist'] == 'bruno mars']

,chartname,date,song,artist,rank,month_yr,asian,name,country,lineage,chart_name
0,weekly_hot_100,2011-01-03,Grenade,bruno mars,1,2011-01,True,bruno mars,Filipino American,Mixed Asian,weekly_hot_100
9,weekly_hot_100,2011-01-03,Just The Way You Are,bruno mars,10,2011-01,True,bruno mars,Filipino American,Mixed Asian,weekly_hot_100
85,weekly_hot_100,2011-01-03,Marry You,bruno mars,86,2011-01,True,bruno mars,Filipino American,Mixed Asian,weekly_hot_100
101,weekly_hot_100,2011-01-10,Grenade,bruno mars,2,2011-01,True,bruno mars,Filipino American,Mixed Asian,weekly_hot_100
108,weekly_hot_100,2011-01-10,Just The Way You Are,bruno mars,9,2011-01,True,bruno mars,Filipino American,Mixed Asian,weekly_hot_100
...,...,...,...,...,...,...,...,...,...,...,...
36339,weekly_hot_100,2017-12-18,That's What I Like,bruno mars,40,2017-12,True,bruno mars,Filipino American,Mixed Asian,weekly_hot_100
36442,weekly_hot_100,2017-12-25,That's What I Like,bruno mars,43,2017-12,True,bruno mars,Filipino American,Mixed Asian,weekly_hot_100
36547,weekly_hot_100,2018-01-01,That's What I Like,bruno mars,48,2018-01,True,bruno mars,Filipino American,Mixed Asian,weekly_hot_100
36631,weekly_hot_100,2018-01-08,That's What I Like,bruno mars,32,2018-01,True,bruno mars,Filipino American,Mixed Asian,weekly_hot_100


In [33]:
# Check Billboard hot 100
b_hot = df_hot_100.groupby(['month_yr', 'name'])['asian'].sum().to_frame()
b_hot = b_hot[b_hot['asian'] != 0].reset_index().set_index('month_yr') 

In [34]:
b_hot.loc['2019-07':'2019-08']

,name,asian
month_yr,,
